In [1]:
import pandas as pd
import librosa 
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
from keras.regularizers import l2


In [2]:
data = pd.read_csv('Data/features_30_sec.csv')
data = data.drop(labels=['filename', 'length'], axis=1)

# X = data.drop('label', axis=1)
# y = data['label']

class_list = data.iloc[:, -1]
convertor = LabelEncoder()
y = convertor.fit_transform(class_list)

fit = StandardScaler()
X = fit.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


def trainModel(model, epochs, optimizer):
    batch_size = 128
    model.compile(optimizer=optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model.fit(X_train, y_train, 
                     validation_data=(X_test, y_test), 
                     epochs=epochs, 
                     batch_size=batch_size)

def plotValidate(history):
    print("Validation Accuracy:", max(history.history['val_accuracy']))
    pd.DataFrame(history.history).plot(figsize=(12,6))
    plt.show()

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax'),
])

print(model.summary())
model_history = trainModel(model=model, epochs=600, optimizer='adam')
plotValidate(model_history)



In [3]:
model.save('genre_classification_model.h5')

model.save_weights('genre_classification.weights.h5')

In [6]:
from keras.models import load_model
from features_extraction import extract_features

new_data = extract_features('Data\genres_original\classical\classical.00000.wav')

new_song_df = pd.DataFrame([new_data])

new_song_scaled = fit.transform(new_song_df)

model = load_model('genre_classification_model.h5')
model.load_weights('genre_classification.weights.h5')

predictions = model.predict(new_song_scaled)

predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Alex\AppData\Local\Temp\ipykernel_17896\1137310181.py:4: SyntaxWarning: invalid escape sequence '\g'
  new_data = extract_features('Data\genres_original\classical\classical.00000.wav')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted class: [9]


c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
